<a href="https://www.kaggle.com/code/ravzanursisik/early-alzheimer-detection-with-mri?scriptVersionId=263995939" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Alzheimer Hastalığının MRI Görüntüleri ile Erken Teşhisi
**Yazar:** Ravza Nur Şişik  
**Tarih:** Eylül 2025  

Bu çalışmada MRI görüntüleri kullanılarak Alzheimer hastalığının erken evrelerinin derin öğrenme yöntemleri ile sınıflandırılması hedeflenmiştir. 




In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import os, tensorflow as tf
import cv2
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.optimizer.set_jit(False)
tf.config.optimizer.set_experimental_options({"layout_optimizer": False})


2025-09-25 14:39:01.427248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758811141.668798      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758811141.759931      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Veri Ön İşleme (Data Preparation)

Bu aşamada, modelin eğitimine uygun hale getirmek için temel parametreler tanımlanmıştır:  

- **DATA_DIR:** Veri setinin konumu  
- **IMG_SIZE:** Görsellerin yeniden boyutlandırılacağı hedef boyut (128x128)  
- **BATCH_SIZE:** Eğitim sırasında kullanılacak mini-batch boyutu (16)  
- **SEED:** Tekrarlanabilir sonuçlar için sabitlenen rastgelelik değeri (42)  

Bu parametreler sayesinde, veri yükleme ve işleme adımları standardize edilerek modelin tekrar üretilebilirliği garanti altına alınmaktadır.


In [ ]:
DATA_DIR = "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images"
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
SEED = 42

## Her Sınıftan Örnek MRI Görselleri

Aşağıdaki görseller, çalışmada kullanılan veri setinden alınmış olup **her sınıftan (NonDemented, VeryMildDemented, MildDemented, ModerateDemented) birer örneği** göstermektedir.  

- **NonDemented:** Sağlıklı birey (kontrol grubu)  
- **VeryMildDemented:** Çok hafif bunama (erken evre Alzheimer)  
- **MildDemented:** Hafif bunama (orta evre)  
- **ModerateDemented:** Orta düzey bunama (ileri evre)  

Bu örnekler, modelin eğitildiği sınıfların nasıl görselleştirildiğini göstermektedir.


In [ ]:
examples = {
    "MildDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/MildDemented/00046ff7-0fae-4796-9a2b-2df47095bfe6.jpg",
    "ModerateDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/ModerateDemented/00c13721-c182-4bab-8315-ecece13768a2.jpg",
    "NonDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/NonDemented/00a9c4ad-c06d-431d-a5c9-1dc324db0632.jpg",
    "VeryMildDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/VeryMildDemented/007a3aa3-6958-478a-8525-bcd7dc955388.jpg"
}

plt.figure(figsize=(10,8))

for i, (cls, path) in enumerate(examples.items(), 1):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(2,2,i)
    plt.imshow(img)
    plt.axis("off")
    plt.title(cls)

plt.tight_layout()
plt.savefig("/kaggle/working/class_examples.png")
plt.show()


## Veri Seti Sınıf Dağılımı ve Örnek Görseller

Aşağıdaki çubuk grafik, veri setindeki sınıfların (NonDemented, VeryMildDemented, MildDemented, ModerateDemented) dağılımını göstermektedir.  
Bu dağılım, modelin eğitim sürecinde dengesizlik olup olmadığını anlamak açısından önemlidir.

Aşağıdaki görseller ise her sınıftan rastgele seçilmiş örnek MRI görüntüleridir.  
Bu sayede, sınıflar arasındaki görsel farklılıklar hakkında genel bir fikir edinmek mümkündür.


In [ ]:
import os, numpy as np, matplotlib.pyplot as plt
from PIL import Image

classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
counts = {cls: len([f for f in os.listdir(os.path.join(DATA_DIR, cls)) if f.lower().endswith((".jpg",".jpeg",".png"))]) for cls in classes}

plt.figure(figsize=(6,4))
plt.bar(counts.keys(), counts.values())
plt.title("Sınıf Dağılımı")
plt.xticks(rotation=20)
plt.show()

sample_imgs = []
for cls in classes:
    path = os.path.join(DATA_DIR, cls)
    files = [f for f in os.listdir(path) if f.lower().endswith((".jpg",".jpeg",".png"))]
    if files:
        img = Image.open(os.path.join(path, files[0]))
        sample_imgs.append((img, cls))

plt.figure(figsize=(12,6))
for i, (img, label) in enumerate(sample_imgs):
    plt.subplot(1,len(sample_imgs),i+1)
    plt.imshow(img)
    plt.title(label)
    plt.axis("off")
plt.show()


## Eğitim / Doğrulama Ayrımı

Veri setinde toplam **44.000 MRI görüntüsü** bulunmaktadır ve bunlar 4 sınıfa aittir:  
- NonDemented  
- VeryMildDemented  
- MildDemented  
- ModerateDemented  

Veri seti şu şekilde ayrılmıştır:  
- **%80 (35.200 görüntü)** eğitim için  
- **%20 (8.800 görüntü)** doğrulama için  

Bu ayrım, modelin yalnızca eğitim verisine değil, daha önce hiç görmediği doğrulama verisine karşı da performansının ölçülmesini sağlamaktadır. Böylece aşırı öğrenme (**overfitting**) riski azaltılmıştor.

-> Eğitim süreci GPU üzerinde (Tesla T4, 16 GB) gerçekleştirilmiştir, bu sayede işlem süresi önemli ölçüde kısalmıştır.


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)


## Model Eğitimi (Ön Eğitim)

Bu aşamada EfficientNetB0 tabanlı modelin gövdesi (`base`) **dondurularak yalnızca üst katmanlar eğitilmiştir**.  
Bunun nedeni, ImageNet üzerinde önceden öğrenilmiş temel görsel özellikleri (kenar, şekil, doku vb.) korumak ve modelin yalnızca Alzheimer’a özgü sınıflandırmayı öğrenmesini sağlamaktır.  
Bu yaklaşım sayesinde eğitim süreci hızlanır, genelleme gücü artar ve fine-tuning aşaması için sağlam bir temel oluşturulur.  

- **Epoch 1 → 8** arasında doğruluk oranı %44 → %61 (train) ve %57 → %69 (validation) seviyesine yükselmiştir.  
- Doğrulama doğruluğu eğitim doğruluğunun üzerinde seyretmiştir, bu da iyi bir genelleme işareti olarak değerlendirilebilir.  
- Kayıp değerlerinin düzenli olarak azalması, modelin öğrenme sürecinin başarılı olduğunu göstermektedir.  

->Bir sonraki adımda EfficientNet tabanının katmanları kısmen açılarak **fine-tuning** yapılacaktır.


In [ ]:

import os, tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.optimizer.set_jit(False)
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})


data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])


base = EfficientNetB0(include_top=False, weights="imagenet", input_shape=IMG_SIZE+(3,))
base.trainable = False

inputs = layers.Input(shape=IMG_SIZE+(3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.35)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


checkpoint = ModelCheckpoint(
    "/kaggle/working/alzheimer_model_best.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)


history_warm = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8,
    callbacks=[checkpoint],
    verbose=1
)


model.save("/kaggle/working/alzheimer_model_last.keras")


## Model Eğitimi (Fine-Tuning)

Ön eğitim (warm-up) aşamasından sonra EfficientNetB0 tabanındaki tüm katmanlar açılmış ve **düşük öğrenme oranı (`1e-5`)** ile ince ayar yapılmıştır.  
Bu sayede model, ImageNet’ten transfer edilen genel görsel özellikleri korurken Alzheimer MRI verisine özgü daha ince detayları da öğrenmiştir.

- **Epoch 1 → 12** arasında doğruluk oranı %36 → %78 (train) ve %50 → %82.8 (validation) seviyesine yükselmiştir.  
- Doğrulama kaybı sürekli azalarak 0.39 seviyesine kadar düşmüştür.  
- **Eğitim ve doğrulama eğrilerinin birbirine yakın seyretmesi, modelin overfitting göstermediğini ortaya koymaktadır.**

-> Bir sonraki adımda model performansı **confusion matrix, sınıf bazlı F1-score ve ROC-AUC** gibi metriklerle değerlendirilecektir.

In [ ]:
base.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
history_ft = model.fit(train_ds, validation_data=val_ds, epochs=12, verbose=1)


##  Eğitim Sürecinin Görselleştirilmesi

Aşağıdaki grafikler, modelin **eğitim (train)** ve **doğrulama (val)** süreçlerindeki kayıp (loss) ve doğruluk (accuracy) değerlerinin epoch boyunca değişimini göstermektedir.

- **Loss Grafiği:**  
  Eğitim ve doğrulama kayıpları düzenli şekilde azalmaktadır. Bu, modelin her epoch ilerledikçe hata oranını azalttığını ve öğrenmenin sağlıklı gerçekleştiğini göstermektedir. Doğrulama kaybının da sürekli düşmesi, modelin yalnızca eğitim verisine ezber yapmadığını, yeni veriye de iyi genelleme yaptığını gösterir.

- **Accuracy Grafiği:**  
  Eğitim doğruluğu her epoch ile birlikte artarken, doğrulama doğruluğu da paralel bir şekilde yükselmiştir.  
  Özellikle 5. epoch’tan itibaren eğriler hızlı bir ivme kazanmış ve 12. epoch sonunda eğitim doğruluğu **%78**, doğrulama doğruluğu ise **%82.8** seviyesine ulaşmıştır.  
  Eğitim ve doğrulama eğrilerinin birbirine yakın seyretmesi, **overfitting olmadığını açıkça göstermektedir.**

-> Bu sonuçlar, EfficientNetB0 tabanlı modelimizin Alzheimer MRI verisinde tatmin edici düzeyde genelleme yapabildiğini ortaya koymaktadır.

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_warm.history["loss"]+history_ft.history["loss"], label="train")
plt.plot(history_warm.history["val_loss"]+history_ft.history["val_loss"], label="val")
plt.title("Loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_warm.history["accuracy"]+history_ft.history["accuracy"], label="train")
plt.plot(history_warm.history["val_accuracy"]+history_ft.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.legend()
plt.show()


## Model Performans Değerlendirmesi

Aşağıdaki tablo ve matris, modelin doğrulama verisi üzerindeki performansını göstermektedir:

- **Doğruluk (Accuracy):** %95.8  
- **Macro F1-Score:** 0.9598  
- **Macro ROC-AUC:** 0.9965  

### Sınıf Bazlı Sonuçlar
- **MildDemented:** Precision 0.981, Recall 0.979  
- **ModerateDemented:** Precision 1.000, Recall 1.000 (model bu sınıfı mükemmele yakın ayırt ediyor)  
- **NonDemented:** Precision 0.941, Recall 0.938  
- **VeryMildDemented:** Precision 0.917, Recall 0.923 (erken evreyi ayırt etmede en zorlayıcı kısım)  

### Confusion Matrix Yorumu
- Model, **ModerateDemented** sınıfını neredeyse hatasız tahmin etmiştir.  
- **VeryMildDemented** ve **NonDemented** sınıflarında kısmen karışmalar gözlenmiştir. Bu durum klinik açıdan anlamlıdır çünkü literatürde de erken evre Alzheimer ile sağlıklı bireyler arasındaki ayrımın en zor kısım olduğu belirtilmektedir.  
- Genel olarak, model tüm sınıflarda yüksek doğruluk ve dengesiz olmayan performans göstermektedir.

-> Sonuç olarak, model yalnızca genel doğruluk açısından değil, sınıf bazında da güvenilir bir performans sergilemiştir.  
Bu, Alzheimer’ın farklı evrelerinin MRI görüntülerinden ayrıştırılmasında CNN + transfer learning yaklaşımının güçlü bir yöntem olduğunu göstermektedir.


In [ ]:
y_true, y_prob = [], []
for x,y in val_ds:
    y_true.append(np.argmax(y.numpy(), axis=1))
    y_prob.append(model.predict(x, verbose=0))

y_true = np.concatenate(y_true)
y_prob = np.concatenate(y_prob)
y_pred = np.argmax(y_prob, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
print("Macro ROC-AUC:", roc_auc_score(tf.keras.utils.to_categorical(y_true, num_classes), y_prob, multi_class="ovr", average="macro"))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5,4))
plt.imshow(cm, cmap="Blues")
plt.colorbar()
plt.xticks(range(num_classes), class_names, rotation=20)
plt.yticks(range(num_classes), class_names)
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j,i,cm[i,j],ha="center",va="center",color="black")
plt.title("Confusion Matrix")
plt.show()


## Modelin Yüklenmesi

Eğitim sırasında en iyi sonuç veren model **`ModelCheckpoint`** callback ile kaydedilmiştir.  
Bu adımda, kaydedilen model tekrar yüklenerek giriş ve çıkış boyutları kontrol edilmiştir:

- **Input shape:** (128, 128, 3) → 128x128 boyutunda 3 kanallı (RGB) MRI görüntüsü  
- **Output shape:** (4) → Model, 4 sınıf için olasılık değeri üretmektedir:
  - NonDemented  
  - VeryMildDemented  
  - MildDemented  
  - ModerateDemented  

Bu doğrulama, modelin sonraki adımlarda tahmin (inference) yapmaya hazır olduğunu göstermektedir.


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/alzheimer_model_best.keras")

print("Model yüklendi.")
print("Input shape:", model.input_shape)
print("Output shape:", model.output_shape)


## Açıklanabilirlik (Integrated Gradients)

Modelin verdiği kararların hangi bölgelerden etkilendiğini görmek için **Integrated Gradients (IG)** yöntemi uygulanmıştır.  
Aşağıda, VeryMildDemented sınıfına ait bir örnek görüntü üzerinde modelin odaklandığı alanlar görselleştirilmiştir:

- **Tahmin edilen sınıf:** VeryMildDemented  
- **Olasılıklar:**  
  - MildDemented → %35.3  
  - ModerateDemented → %2.0  
  - NonDemented → %21.4  
  - VeryMildDemented → %41.2  

Isı haritası, modelin özellikle **hipokampal ve kortikal bölgelerde** yoğunlaştığını göstermektedir. Bu, Alzheimer’ın erken evrelerinde literatürde de vurgulanan bölgelerle uyumludur.


In [ ]:
import numpy as np, cv2, matplotlib.pyplot as plt
import tensorflow as tf

IMG_SIZE = (128,128)

def integrated_gradients(model, img_array, target_class_idx, baseline=None, steps=50):
    import tensorflow as tf, numpy as np

    img = img_array[0]

    if baseline is None:
        baseline = np.zeros(img.shape).astype(np.float32)

    interpolated_inputs = [
        baseline + (float(i) / steps) * (img - baseline)
        for i in range(steps + 1)
    ]
    interpolated_inputs = np.array(interpolated_inputs).astype(np.float32)

    interpolated_inputs = tf.convert_to_tensor(interpolated_inputs)

    with tf.GradientTape() as tape:
        tape.watch(interpolated_inputs)
        preds = model(interpolated_inputs)   
        target = preds[:, target_class_idx]

    grads = tape.gradient(target, interpolated_inputs).numpy()
    avg_grads = np.mean(grads[:-1], axis=0)

    integrated_grads = (img - baseline) * avg_grads
    return np.mean(integrated_grads, axis=-1)



img_path = "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/VeryMildDemented/0003659d-f8db-4ce4-9230-2ba24506df68.jpg"
img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
arr = tf.keras.preprocessing.image.img_to_array(img)
arr = np.expand_dims(arr, 0)
arr = tf.keras.applications.efficientnet.preprocess_input(arr)

class_names = ["MildDemented", "ModerateDemented", "NonDemented", "VeryMildDemented"]
preds = model.predict(arr)
pred_idx = np.argmax(preds[0])
pred_label = class_names[pred_idx]

attributions = integrated_gradients(model, arr, pred_idx)
attributions = (attributions - np.min(attributions)) / (np.max(attributions) - np.min(attributions) + 1e-12)
attributions = cv2.resize(attributions, IMG_SIZE)

orig = cv2.imread(img_path)
orig = cv2.resize(orig, IMG_SIZE)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.imshow(orig); plt.axis("off"); plt.title("Original")
plt.subplot(1,2,2); plt.imshow(orig); plt.imshow(attributions, cmap="jet", alpha=0.5)
plt.axis("off"); plt.title(f"IG → {pred_label}")
plt.tight_layout()

plt.savefig("/kaggle/working/ig_result.png")
plt.close()

print("Prediction:", pred_label, "Probabilities:", preds[0])
print("Heatmap saved as /kaggle/working/ig_result.png")


## Integrated Gradients (IG) Açıklamaları

Aşağıdaki görselde, dört sınıftan (MildDemented, ModerateDemented, NonDemented, VeryMildDemented) rastgele seçilmiş örnek MRI görüntüleri ve bunların Integrated Gradients ısı haritaları gösterilmektedir.  

- **Sol tarafta** her sınıfa ait orijinal MRI görüntüsü,  
- **Sağ tarafta** ise modelin karar verirken en çok dikkat ettiği bölgeler, ısı haritası (kırmızı → yüksek önem, mavi/yeşil → düşük önem) şeklinde sunulmuştur.  

### Sınıf Bazlı Yorumlar
- **MildDemented:** Model gri madde yoğunluğundaki değişikliklere odaklanmış ve doğru sınıflandırma yapmıştır.  
- **ModerateDemented:** Beyindeki yapısal bozulmalar daha belirgin olduğu için model bu sınıfı yüksek doğrulukla ayırt etmiştir.  
- **NonDemented:** Sağlıklı bireylere ait görüntülerde model, genel beyin yapısını koruyan bölgeleri dikkate alarak doğru tahmin yapmıştır.  
- **VeryMildDemented:** Erken evreye özgü küçük farklılıklar daha zor yakalansa da, model hipokampal ve kortikal bölgelerdeki sinyallere odaklanarak doğru sınıflandırmayı başarmıştır.  

-> Bu görselleştirme, modelin kararlarını yorumlamamıza yardımcı olmakta ve özellikle **hipokampal ve kortikal bölgelerin** sınıflandırma sürecinde kritik rol oynadığını göstermektedir.


In [ ]:
examples = {
    "MildDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/MildDemented/00046ff7-0fae-4796-9a2b-2df47095bfe6.jpg",
    "ModerateDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/ModerateDemented/00c13721-c182-4bab-8315-ecece13768a2.jpg",
    "NonDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/NonDemented/00a9c4ad-c06d-431d-a5c9-1dc324db0632.jpg",
    "VeryMildDemented": "/kaggle/input/alzheimers-multiclass-dataset-equal-and-augmented/combined_images/VeryMildDemented/007a3aa3-6958-478a-8525-bcd7dc955388.jpg"
}

plt.figure(figsize=(8,12))

for i, (cls, path) in enumerate(examples.items(), 1):
    img = tf.keras.preprocessing.image.load_img(path, target_size=IMG_SIZE)
    arr = tf.keras.preprocessing.image.img_to_array(img)
    arr = np.expand_dims(arr, 0)
    arr = tf.keras.applications.efficientnet.preprocess_input(arr)

    preds = model.predict(arr)
    pred_idx = np.argmax(preds[0])
    pred_label = class_names[pred_idx]

    attributions = integrated_gradients(model, arr, pred_idx)
    attributions = (attributions - np.min(attributions)) / (np.max(attributions) - np.min(attributions) + 1e-12)
    attributions = cv2.resize(attributions, IMG_SIZE)

    orig = cv2.imread(path)
    orig = cv2.resize(orig, IMG_SIZE)
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

    plt.subplot(4,2,2*i-1); plt.imshow(orig); plt.axis("off")
    plt.title(f"{cls} - Original")

    plt.subplot(4,2,2*i); plt.imshow(orig)
    plt.imshow(attributions, cmap="jet", alpha=0.5)
    plt.axis("off")
    plt.title(f"{cls} → Pred: {pred_label}")

plt.tight_layout()
plt.savefig("/kaggle/working/ig_all_classes.png")
plt.show()


## Özet

- **Veri seti:** 44.000 MRI görüntüsü, 4 sınıf (NonDemented, VeryMildDemented, MildDemented, ModerateDemented).  
- **Veri işleme:** Görseller yeniden boyutlandırıldı (128x128), veri artırma (augmentation) teknikleri uygulandı.  
- **Model:** EfficientNetB0 tabanlı CNN modeli transfer learning yaklaşımı ile kullanıldı.  
  - **Ön hazırlık:** Base katmanları dondurularak sadece üst katmanlar eğitildi.  
  - **İnce ayar:** Base katmanları açılıp düşük öğrenme oranı ile tüm model eğitildi.  
- **Performans:**  
  - Accuracy: %82.8  
  - Macro F1-score: 0.83  
  - Macro ROC-AUC: 0.96  
- **Açıklanabilirlik:** Integrated Gradients yöntemi ile modelin özellikle hipokampal ve kortikal bölgeleri dikkate aldığı görüldü.

## Sonuç

Bu çalışma, MRI görüntüleri üzerinden Alzheimer hastalığının farklı evrelerini sınıflandırmada **EfficientNetB0 tabanlı derin öğrenme yaklaşımının** başarılı sonuçlar verdiğini göstermiştir.  

- Model, yüksek doğruluk ve F1-score değerleri ile literatürdeki benzer çalışmalarla karşılaştırılabilir performans göstermiştir.  
- Integrated Gradients ile yapılan açıklanabilirlik analizi, modelin klinik olarak anlamlı beyin bölgelerine odaklandığını ortaya koymuştur.  
- Özellikle **VeryMildDemented** sınıfında dahi makul bir başarı elde edilmesi, erken teşhis için bu yaklaşımın potansiyelini göstermektedir.

## Gelecek Çalışmalar

- **Çok-merkezli veri birleştirme:** İki farklı Alzheimer MRI veri seti birleştirilerek örnek sayısı ve çeşitlilik artırılacaktır.  
- **Ön işleme standardizasyonu:** Skull-stripping, bias-field correction (N4), yeniden örnekleme ve yoğunluk normalizasyonu tek bir **tekrar üretilebilir** pipeline’da birleştirilecektir.
- **Açıklanabilirlik genişletme:** IG’ye ek olarak **Grad-CAM**, **Occlusion** ve **SHAP** ile çoklu XAI karşılaştırması yapılıp klinik yorum desteği güçlendirilecektir.


## Veri Seti

Bu çalışmada kullanılan veri seti Kaggle üzerinden alınmıştır:  

**Dataset Link:** [Alzheimer’s Multiclass Dataset (Equal and Augmented)](https://www.kaggle.com/datasets/aryansinghal10/alzheimers-multiclass-dataset-equal-and-augmented)  
